In [1]:
import boto3
from botocore.config import Config
from datetime import datetime, timedelta

In [4]:
session = boto3.Session(
    aws_access_key_id="1c407b86-9d98-4a01-bf59-3b45eed92548",
    aws_secret_access_key="EuwKhItrKdGMxjTLTrltOOX5aJQ0ozpp",
)

In [5]:
s3 = session.client(
    "s3",
    endpoint_url="https://files.polygon.io",
    config=Config(signature_version="s3v4"),
)

In [6]:
paginator = s3.get_paginator("list_objects_v2")

In [9]:
prefix = "us_stocks_sip"
bucket = "flatfiles"

In [186]:
allfilenames = []

for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
    for obj in page["Contents"]:
        allfilenames.append(obj["Key"])

In [194]:
category = [i.split("/")[1] for i in allfilenames]
stonkfilenames = [i for i in allfilenames if i.split("/")[1] == "day_aggs_v1"]

In [201]:
datestrings = [i.split("/")[-1].split(".")[0] for i in allfilenames]
date = [datetime.date(datetime.strptime(i, "%Y-%m-%d")) for i in datestrings]

In [198]:
current_date = datetime.now()
five_years_ago = current_date - timedelta(days=365*5)
date_five_years_ago = five_years_ago.date()

In [207]:
recent = [i for i,j in zip(stonkfilenames, date) if j > date_five_years_ago]

In [223]:
for i in recent:
    localfilename = "day_aggs/" + i.split("/")[-1]

    try:
        s3.download_file(bucket, i, localfilename)
    except Exception as e:
        print(e)